In [30]:
import networkx as nx
from pyvis.network import Network
import random
import os

In [31]:

def load_random_graph(directory):
    files = [file for file in os.listdir(directory) if file.endswith('.graphml')] # or other format
    random_file = random.choice(files)
    path = os.path.join(directory, random_file)
    print(f'Loading graph from {path}')
    return nx.read_graphml(path)  # Change this according to your graph format

def get_root_nodes(G):
    return [n for n, d in G.in_degree() if d == 0]

def plot_graph_with_sccs(G, sccs):
    net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', notebook=True)

    # Add nodes and edges
    for node in G.nodes:
        net.add_node(node, label=str(node), title=str(node))

    for edge in G.edges:
        net.add_edge(edge[0], edge[1])

    #loop over all sccs, select a unique color for each scc and color the nodes in the scc
    colors = ['#'+str(hex(random.randint(0, 16777215)))[2:] for i in range(len(sccs))]
    for i, scc in enumerate(sccs):
        for node in scc:
            net.get_node(node)['color'] = colors[i]
            


    # Generate network with specific layout
    net.from_nx(G)
    net.show('graph_viz_sccs.html')

def plot_graph_with_roots(G, roots):
    net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', notebook=True)

    # Add nodes and edges
    for node in G.nodes:
        net.add_node(node, label=str(node), title=str(node))

    for edge in G.edges:
        net.add_edge(edge[0], edge[1])

    for node in roots:
        net.get_node(node)['color'] = 'red'
            
    # Targets
    for node, attributes in G.nodes(data=True):
        if attributes['cat'] == '1':
            net.get_node(node)['color'] = 'green'

    # Generate network with specific layout
    net.from_nx(G)
    net.show('graph_viz_sccs.html')

def get_target_nodes(root, G):
    target_nodes = [node for node, attributes in G.nodes(data=True) if attributes['cat'] == '1']
    return target_nodes





In [32]:
directory = '/home/cyril/ssh-rlkex/Generated_Graphs/output/basic/V_6_8_P1/24'
G = load_random_graph(directory)
#compute all sccs
sccs = list(nx.strongly_connected_components(G))



Loading graph from /home/cyril/ssh-rlkex/Generated_Graphs/output/basic/V_6_8_P1/24/28017-1643890740.graphml


In [33]:
#Detect if there are any cycles
if len(sccs) == len(G.nodes):
    print("No cycles detected")
else:
    print("Cycles detected")

Cycles detected


In [34]:
#Compare the number of nodes in the sccs with the number of nodes in the graph
scc_nodes = [node for scc in sccs for node in scc]
print(f'Number of nodes in the graph: {len(G.nodes)}')
print(f'Number of nodes in the sccs: {len(scc_nodes)}')
print(f'Number of SCCs: {len(sccs)}')

Number of nodes in the graph: 897
Number of nodes in the sccs: 897
Number of SCCs: 786


In [35]:
#For each scc, check if it contains a node with no incoming edges to the scc
#this node is the root node of the scc

def get_root_nodes_from_scc(sccs, G):
    """If we consider each scc as a node, then we want to get the sccs that have no incoming edges."""
    root_nodes = []
    for scc in sccs:
        for node in scc:
            if len([n for n in G.predecessors(node) if n not in scc]) == 0:
                #append the first node of the scc that has no incoming edges
                #transform the scc to list
                root_nodes.extend(list(scc))
                break
    return root_nodes

root_nodes = get_root_nodes_from_scc(sccs, G)
print(root_nodes)
plot_graph_with_roots(G, root_nodes)

['n23', 'n92', 'n86', 'n5', 'n12', 'n20', 'n392', 'n56', 'n57', 'n243', 'n232', 'n61', 'n43', 'n91', 'n82', 'n401', 'n250', 'n8', 'n231', 'n397', 'n25', 'n236', 'n32', 'n242', 'n31', 'n78', 'n7', 'n244', 'n44', 'n3', 'n230', 'n58', 'n83', 'n37', 'n241', 'n229', 'n247', 'n54', 'n240', 'n98', 'n79', 'n13', 'n245', 'n400', 'n81', 'n29', 'n60', 'n18', 'n94', 'n24', 'n97', 'n395', 'n36', 'n48', 'n59', 'n62', 'n50', 'n17', 'n88', 'n95', 'n34', 'n403', 'n2', 'n234', 'n55', 'n26', 'n15', 'n6', 'n19', 'n14', 'n238', 'n399', 'n33', 'n16', 'n1', 'n80', 'n248', 'n0', 'n396', 'n398', 'n239', 'n87', 'n10', 'n45', 'n47', 'n246', 'n90', 'n35', 'n11', 'n89', 'n9', 'n249', 'n84', 'n49', 'n96', 'n4', 'n93', 'n237', 'n85', 'n63', 'n391', 'n233', 'n27', 'n30', 'n402', 'n51', 'n235', 'n46', 'n28', 'n404', 'n405', 'n406', 'n407', 'n408', 'n409', 'n410', 'n411', 'n412', 'n413', 'n414', 'n415', 'n416', 'n417', 'n418', 'n419', 'n420', 'n421', 'n422', 'n423', 'n424', 'n425', 'n426', 'n427', 'n428', 'n429', 'n710

In [36]:
#Loop over all root nodes and make sure that every possible nodes from the graph, not just targets, is reachable from at least one root node
#If there is a node that is not reachable from any root node, then there is a problem

nb_of_reachable_nodes = 0

#loop over all roots, do a bfs from each root and mark all visited nodes, then sum the number of visited nodes
visited_nodes = {}
for root in root_nodes:
    visited_nodes[root] = True
    for node in nx.bfs_tree(G, root).nodes:
        visited_nodes[node] = True

nb_of_reachable_nodes = len(visited_nodes)
print(f'Number of reachable nodes: {nb_of_reachable_nodes}')
print(f'Number of nodes in the graph: {len(G.nodes)}')


Number of reachable nodes: 897
Number of nodes in the graph: 897


In [37]:
#for all roots, get the number of reachable targets
target_nodes = get_target_nodes(root_nodes, G)
nb_of_reachable_targets = {}
for root in root_nodes:
    nb_of_reachable_targets[root] = 0
    for node in nx.bfs_tree(G, root).nodes:
        if node in target_nodes:
            nb_of_reachable_targets[root] += 1
            
#print all roots with non zero number of reachable targets
for root in root_nodes:
    if nb_of_reachable_targets[root] > 0:
        print(f'Root {root} has {nb_of_reachable_targets[root]} reachable targets')
#print the total number of reachable targets
print(f'Total number of reachable targets: {sum(nb_of_reachable_targets.values())}')
#print the total number of targets
print(f'Total number of targets: {len(target_nodes)}')
#print the number of root nodes that reach at least one target
print(f'Number of root nodes that reach at least one target: {len([root for root in root_nodes if nb_of_reachable_targets[root] > 0])}')



Root n883 has 6 reachable targets
Root n881 has 6 reachable targets
Total number of reachable targets: 12
Total number of targets: 6
Number of root nodes that reach at least one target: 2


In [28]:
plot_graph_with_sccs(G, sccs)

graph_viz_sccs.html


In [38]:
plot_graph_with_roots(G, root_nodes)

graph_viz_sccs.html


In [11]:
import json
G = load_random_graph(directory)
nb_nodes_before = len(G.nodes)
# take all root nodes and the targets, and only keep the nodes that are on the path between them
root_nodes = get_root_nodes(G)
target_nodes = get_target_nodes(root_nodes, G)
print('Root nodes: ', root_nodes)
print('Target nodes: ', target_nodes)
nodes_to_keep = []
for root in root_nodes:
    for target in target_nodes:
        print(f'Computing shortest path between {root} and {target}')
        path = nx.shortest_path(G, root, target)
        nodes_to_keep.extend(path)
G = G.subgraph(nodes_to_keep)

#print number of kept nodes over total number of nodes
print('Kept nodes: ', len(nodes_to_keep))
print('Total nodes: ', nb_nodes_before)


# Convert the graph to a format that Cytoscape.js can understand
cytoscape_json = nx.cytoscape_data(G)['elements']

# Save the graph data to a JSON file
with open('graph_data.json', 'w') as f:
    json.dump(cytoscape_json, f)

Loading graph from /home/cyril/ssh-rlkex/Generated_Graphs/output/basic/V_6_8_P1/24/28757-1643890740.graphml
Root nodes:  ['n0', 'n404', 'n405', 'n406', 'n407', 'n408', 'n409', 'n410', 'n411', 'n412', 'n413', 'n414', 'n415', 'n416', 'n417', 'n418', 'n419', 'n420', 'n421', 'n422', 'n423', 'n424', 'n425', 'n426', 'n427', 'n428', 'n429', 'n700', 'n704', 'n708', 'n709', 'n710', 'n718', 'n719', 'n720', 'n721', 'n722', 'n723', 'n729', 'n733', 'n751', 'n753', 'n759', 'n769', 'n794', 'n796', 'n799', 'n800', 'n801', 'n802', 'n803', 'n842', 'n843', 'n844', 'n845', 'n846', 'n847', 'n848', 'n849', 'n850', 'n851', 'n852', 'n856', 'n857', 'n858', 'n859', 'n861', 'n863', 'n865', 'n867', 'n872', 'n879', 'n888', 'n889', 'n890', 'n891', 'n892', 'n893']
Target nodes:  ['n868', 'n869', 'n870', 'n873', 'n874', 'n876']
Computing shortest path between n0 and n868


NetworkXNoPath: No path between n0 and n868.